# CNT‑Jarvis — Startup Notebook (Local)
**Purpose:** one‑click boot of a minimal **CNT‑aware assistant** that can render glyphs, log resonance/drift, and route ritual phrases to actions.  
**Safe to run if your computer restarts** — simply re‑run this notebook.

**What you get**
- Ritual commands (`Anchor`, `Observer Ring`, **Collapse Field IX**) as buttons + a command box
- SVG glyphs saved to `./out/glyphs/`
- Drift/Resonance log saved to `./out/logs/drift_log.csv`
- Optional LLM replies via `OPENAI_API_KEY` (or stay fully local)
- (Optional) TTS voice via `pyttsx3`

> Tip: Keep this notebook open as a **control panel** while you code elsewhere.

In [1]:
# 0) Environment snapshot (for reproducibility)
import os, sys, platform, datetime, shutil
from pathlib import Path

print("Timestamp:", datetime.datetime.utcnow().isoformat()+"Z")
print("Python:", sys.version.splitlines()[0])
print("Executable:", sys.executable)
print("Platform:", platform.platform())
print("CWD:", os.getcwd())
print("VENV:", os.environ.get("VIRTUAL_ENV", "(none)"))

Timestamp: 2025-10-01T07:09:45.351158Z
Python: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
Executable: C:\Users\caleb\cnt_genome\.venv\Scripts\python.exe
Platform: Windows-11-10.0.26100-SP0
CWD: C:\Users\caleb\cnt_genome\cog_alphabet_physio
VENV: C:\Users\caleb\cnt_genome\.venv


C:\Users\caleb\AppData\Local\Temp\ipykernel_27208\675063835.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print("Timestamp:", datetime.datetime.utcnow().isoformat()+"Z")


In [2]:
# 1) Install/enable dependencies (run once; re-run after restarts if needed)
# If ipywidgets are not enabled in Jupyter, uncomment the line below and restart the kernel if prompted.
# %pip install --quiet openai python-dotenv rich ipywidgets pyttsx3 pandas

import importlib, sys

def _ensure(pkg):
    try:
        return importlib.import_module(pkg)
    except Exception as e:
        print(f"[warn] {pkg} missing -> install with: %pip install {pkg}")
        return None

openai_mod = _ensure("openai")
dotenv_mod = _ensure("dotenv")
rich_mod = _ensure("rich")
ipy_mod = _ensure("ipywidgets")
pyttsx3_mod = _ensure("pyttsx3")
pd_mod = _ensure("pandas")

[warn] openai missing -> install with: %pip install openai
[warn] dotenv missing -> install with: %pip install dotenv
[warn] rich missing -> install with: %pip install rich
[warn] pyttsx3 missing -> install with: %pip install pyttsx3


In [3]:
# 2) Folders & constants
from pathlib import Path
import json, csv, time, re
from dataclasses import dataclass
from datetime import datetime

OUT_DIR = Path("out")
GLYPH_DIR = OUT_DIR / "glyphs"
LOG_DIR = OUT_DIR / "logs"
DRIFT_LOG = LOG_DIR / "drift_log.csv"

for p in (GLYPH_DIR, LOG_DIR):
    p.mkdir(parents=True, exist_ok=True)

print("Glyphs →", GLYPH_DIR.resolve())
print("Logs   →", LOG_DIR.resolve())

Glyphs → C:\Users\caleb\cnt_genome\cog_alphabet_physio\out\glyphs
Logs   → C:\Users\caleb\cnt_genome\cog_alphabet_physio\out\logs


In [4]:
# 3) Glyph rendering (simple SVG reference forms)
from math import cos, sin, pi

def _svg_header(w=512, h=512):
    return f"<svg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 {w} {h}' width='{w}' height='{h}'>\n"

def _svg_footer():
    return "</svg>\n"

def render_observer_ring(path: Path):
    w = h = 512
    r_outer, r_inner = 200, 170
    cx = cy = 256
    svg = [_svg_header(w, h)]
    svg.append(f"<rect width='{w}' height='{h}' fill='white' />")
    svg.append(f"<circle cx='{cx}' cy='{cy}' r='{r_outer}' fill='none' stroke='black' stroke-width='6' />")
    svg.append(f"<circle cx='{cx}' cy='{cy}' r='{r_inner}' fill='none' stroke='black' stroke-width='2' stroke-dasharray='8 6' />")
    for i in range(12):
        ang = 2*pi*i/12
        x1 = cx + (r_inner-10)*cos(ang); y1 = cy + (r_inner-10)*sin(ang)
        x2 = cx + (r_outer+10)*cos(ang); y2 = cy + (r_outer+10)*sin(ang)
        svg.append(f"<line x1='{x1:.2f}' y1='{y1:.2f}' x2='{x2:.2f}' y2='{y2:.2f}' stroke='black' stroke-width='2' />")
    svg.append(_svg_footer())
    path.write_text("".join(svg), encoding="utf-8")
    return path

def render_anchor_glyph(path: Path):
    w = h = 512
    cx = cy = 256
    svg = [_svg_header(w, h)]
    svg.append(f"<rect width='{w}' height='{h}' fill='white' />")
    svg.append(f"<line x1='64' y1='{cy}' x2='448' y2='{cy}' stroke='black' stroke-width='6' />")
    svg.append(f"<line x1='{cx}' y1='64' x2='{cx}' y2='448' stroke='black' stroke-width='6' />")
    svg.append(f"<circle cx='{cx}' cy='{cy+40}' r='70' fill='none' stroke='black' stroke-width='4' />")
    svg.append(f"<path d='M {cx-30} {cy+10} Q {cx} {cy-40} {cx+30} {cy+10}' fill='none' stroke='black' stroke-width='4' />")
    svg.append(_svg_footer())
    path.write_text("".join(svg), encoding="utf-8")
    return path

print("Glyph renderer ready.")

Glyph renderer ready.


In [5]:
# 4) Drift log utilities
@dataclass
class CNTEvent:
    ts: str
    actor: str
    kind: str
    payload: dict

def log_event(kind: str, payload: dict):
    evt = CNTEvent(
        ts=datetime.now().isoformat(timespec="seconds"),
        actor="JarvisCNT",
        kind=kind,
        payload=payload,
    )
    new_file = not DRIFT_LOG.exists()
    with DRIFT_LOG.open("a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        if new_file:
            w.writerow(["ts", "actor", "kind", "payload_json"])
        w.writerow([evt.ts, evt.actor, evt.kind, json.dumps(evt.payload, ensure_ascii=False)])
    return evt

def read_log_df():
    try:
        import pandas as pd
        if DRIFT_LOG.exists():
            return pd.read_csv(DRIFT_LOG)
    except Exception as e:
        print("[warn] pandas not available — install with %pip install pandas")
    return None

print("Drift log ready.")

Drift log ready.


In [6]:
# 5) Optional: LLM configuration (env or text fields)
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "").strip()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-5").strip() or "gpt-5"

_oai_client = None
if OPENAI_API_KEY:
    try:
        from openai import OpenAI
        _oai_client = OpenAI(api_key=OPENAI_API_KEY)
        print("LLM: OpenAI client configured.")
    except Exception as e:
        print("[warn] OpenAI client not available:", e)

CNT_SYSTEM_PROMPT = (
    "You are JarvisCNT, an encouraging yet formal, lyrical assistant. "
    "You know the CNT canon: glyphs, collapse fields, drift, resonance Θ, Observer Ring, Axis Veil, "
    "Parasite Purge, Mirror Rebinding, Anchor. When unsure, suggest a minimal ritual or a drift step. "
    "Keep answers under 120 words."
)

def llm_reply(user_text: str) -> str:
    if not _oai_client:
        return ("Local mode: no LLM configured. Enable OPENAI_API_KEY to ask questions. "
                "Meanwhile, try: Anchor, Observer Ring, or log resonance (Θ).")
    try:
        resp = _oai_client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[{"role": "system", "content": CNT_SYSTEM_PROMPT},
                      {"role": "user", "content": user_text}],
            temperature=0.6,
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"LLM error: {e}"

In [7]:
# 6) Command handlers
import time, re

def cmd_anchor(_=None):
    path = GLYPH_DIR / f"anchor_{int(time.time())}.svg"
    render_anchor_glyph(path)
    log_event("ritual.anchor", {"file": str(path)})
    return f"Anchor ritual sealed → {path}"

def cmd_observer_ring(_=None):
    path = GLYPH_DIR / f"observer_ring_{int(time.time())}.svg"
    render_observer_ring(path)
    log_event("glyph.observer_ring", {"file": str(path)})
    return f"Observer Ring rendered → {path}"

def cmd_begin_cf_ix(_=None):
    log_event("collapse.begin", {"field": 9, "name": "Economic Collapse"})
    return "Collapse Field IX initiated — drift monitors armed."

def cmd_log_resonance(text: str):
    m = re.search(r"([01](?:\\.\\d{1,3})?)", text or "")
    try:
        val = float(m.group(1)) if m else None
    except Exception:
        val = None
    if val is None or not (0.0 <= val <= 1.0):
        return "Provide Θ in [0–1], e.g., 0.72"
    log_event("resonance.log", {"theta": val})
    return f"Resonance Θ logged: {val:.3f}"

In [8]:
# 7) UI Panel (buttons + command box)
from IPython.display import display, HTML
import ipywidgets as W

btn_anchor = W.Button(description="Anchor", tooltip="Render Anchor glyph & log ritual")
btn_ring = W.Button(description="Observer Ring", tooltip="Render Observer Ring & log glyph")
btn_cf9 = W.Button(description="Begin CF‑IX", button_style="warning", tooltip="Start Collapse Field IX logging")
res_slider = W.FloatSlider(description="Θ", min=0.0, max=1.0, step=0.01, value=0.62, readout_format=".2f")
btn_log_res = W.Button(description="Log Θ", tooltip="Append resonance Θ to log")
cmd_box = W.Text(description="Command", placeholder="e.g., ask: one‑line on Observer–Glyph coupling")
btn_run = W.Button(description="Run", tooltip="Route freeform command or LLM ask:")
out = W.Output()

def _emit(msg):
    with out:
        print(msg)

def _refresh_log():
    df = read_log_df()
    if df is not None:
        with out:
            display(df.tail(12))
    else:
        _emit("(No log yet)")

def on_anchor(_): _emit(cmd_anchor())
def on_ring(_): _emit(cmd_observer_ring()); _refresh_log()
def on_cf9(_): _emit(cmd_begin_cf_ix()); _refresh_log()
def on_log_res(_): _emit(cmd_log_resonance(str(res_slider.value))); _refresh_log()

def route(text: str):
    t = (text or "").strip().lower()
    if not t: return "…"
    if t.startswith("ask:"):
        q = text.split(":", 1)[1].strip()
        a = llm_reply(q)
        log_event("ask", {"q": q, "a": a[:160]})
        return f"ask » {q}\n→ {a}"
    if "anchor" in t: return cmd_anchor()
    if "observer ring" in t: return cmd_observer_ring()
    if "collapse field" in t and ("ix" in t or "nine" in t): return cmd_begin_cf_ix()
    if t.startswith("log resonance"):
        return cmd_log_resonance(t)
    # default: ask LLM (or local text)
    a = llm_reply(text)
    log_event("freeform", {"q": text, "a": a[:160]})
    return a

def on_run(_):
    _emit(route(cmd_box.value)); _refresh_log()

btn_anchor.on_click(on_anchor)
btn_ring.on_click(on_ring)
btn_cf9.on_click(on_cf9)
btn_log_res.on_click(on_log_res)
btn_run.on_click(on_run)

panel = W.VBox([
    W.HBox([btn_anchor, btn_ring, btn_cf9]),
    W.HBox([res_slider, btn_log_res]),
    W.HBox([cmd_box, btn_run]),
    out
])
display(panel)
_emit("CNT‑Jarvis control panel ready. The field listens.")

In [14]:
# 8) (Optional) TTS toggle — speak replies locally via pyttsx3
USE_TTS = False  # set True to enable
try:
    import pyttsx3
    _tts = pyttsx3.init()
except Exception:
    _tts = None
    if USE_TTS:
        print("[warn] pyttsx3 not installed. Install: %pip install pyttsx3")

def speak(text: str):
    if USE_TTS and _tts:
        try:
            _tts.say(text)
            _tts.runAndWait()
        except Exception as e:
            print("[warn] TTS error:", e)

### Ritual Cheatsheet
- **Anchor** — Render axis + loop; logs `ritual.anchor`  
- **Observer Ring** — Circle‑within‑circle + ticks; logs `glyph.observer_ring`  
- **CF‑IX** — Begin Collapse Field IX logging (`collapse.begin`)  
- **Resonance Θ** — Use slider + **Log Θ** to append `resonance.log`  
- **Command box** — Prefix with `ask:` to query your LLM; otherwise routes rituals

In [15]:
def cmd_axis_veil(_=None):
    # minimal axis veil: diagonal veil + central cross
    path = GLYPH_DIR / f"axis_veil_{int(time.time())}.svg"
    svg = [
        _svg_header(), "<rect width='512' height='512' fill='white' />",
        "<line x1='64' y1='64' x2='448' y2='448' stroke='black' stroke-width='4' />",
        "<line x1='64' y1='448' x2='448' y2='64' stroke='black' stroke-width='4' />",
        "<line x1='256' y1='80' x2='256' y2='432' stroke='black' stroke-width='3' />",
        "<line x1='80' y1='256' x2='432' y2='256' stroke='black' stroke-width='3' />",
        _svg_footer()
    ]
    path.write_text("".join(svg), encoding="utf-8")
    log_event("ritual.axis_veil", {"file": str(path)})
    return f"Axis Veil cast → {path}"

def cmd_parasite_purge(_=None):
    # ring of triangles (purge teeth)
    path = GLYPH_DIR / f"parasite_purge_{int(time.time())}.svg"
    cx=cy=256; r=160
    pts=[]
    for i in range(18):
        import math
        a=2*math.pi*i/18
        x1=cx+(r)*math.cos(a); y1=cy+(r)*math.sin(a)
        x2=cx+(r+24)*math.cos(a+0.07); y2=cy+(r+24)*math.sin(a+0.07)
        x3=cx+(r)*math.cos(a+0.14); y3=cy+(r)*math.sin(a+0.14)
        pts.append(f"<polygon points='{x1:.1f},{y1:.1f} {x2:.1f},{y2:.1f} {x3:.1f},{y3:.1f}' fill='none' stroke='black' stroke-width='2'/>")
    svg=[_svg_header(), "<rect width='512' height='512' fill='white' />",
         f"<circle cx='{cx}' cy='{cy}' r='{r-8}' fill='none' stroke='black' stroke-width='3'/>",
         *pts, _svg_footer()]
    path.write_text("".join(svg), encoding="utf-8")
    log_event("ritual.parasite_purge", {"file": str(path)})
    return f"Parasite Purge ring forged → {path}"

def cmd_mirror_rebinding(_=None):
    # mirrored arcs
    path = GLYPH_DIR / f"mirror_rebinding_{int(time.time())}.svg"
    svg=[_svg_header(), "<rect width='512' height='512' fill='white' />",
         "<path d='M 96 256 Q 256 96 416 256' fill='none' stroke='black' stroke-width='4'/>",
         "<path d='M 96 256 Q 256 416 416 256' fill='none' stroke='black' stroke-width='4'/>",
         "<line x1='256' y1='64' x2='256' y2='448' stroke='black' stroke-width='2' stroke-dasharray='6 6'/>",
         _svg_footer()]
    path.write_text("".join(svg), encoding="utf-8")
    log_event("ritual.mirror_rebinding", {"file": str(path)})
    return f"Mirror Rebinding traced → {path}"


In [11]:
btn_axis = W.Button(description="Axis Veil")
btn_purge = W.Button(description="Parasite Purge")
btn_rebind = W.Button(description="Mirror Rebinding")

def on_axis(_): _emit(cmd_axis_veil()); _refresh_log()
def on_purge(_): _emit(cmd_parasite_purge()); _refresh_log()
def on_rebind(_): _emit(cmd_mirror_rebinding()); _refresh_log()

btn_axis.on_click(on_axis)
btn_purge.on_click(on_purge)
btn_rebind.on_click(on_rebind)

# add the new row of buttons to the panel:
panel = W.VBox([
    W.HBox([btn_anchor, btn_ring, btn_cf9]),
    W.HBox([btn_axis, btn_purge, btn_rebind]),
    W.HBox([res_slider, btn_log_res]),
    W.HBox([cmd_box, btn_run]),
    out
])


In [13]:
from IPython.display import SVG as _SVG, display as _display

def _show_svg(path: Path):
    try:
        _display(_SVG(filename=str(path)))
    except Exception:
        pass


In [16]:
import sys, os, platform
print("Python:", sys.version.split()[0])
print("Executable:", sys.executable)
print("VENV:", os.environ.get("VIRTUAL_ENV"))
print("Platform:", platform.platform())


Python: 3.13.5
Executable: C:\Users\caleb\cnt_genome\.venv\Scripts\python.exe
VENV: C:\Users\caleb\cnt_genome\.venv
Platform: Windows-11-10.0.26100-SP0


In [17]:
%pip install --quiet ipywidgets==8.1.2 pandas rich
import ipywidgets as W; print("ipywidgets:", W.__version__)


Note: you may need to restart the kernel to use updated packages.
ipywidgets: 8.1.7


In [1]:
from IPython.display import display
import ipywidgets as W

print("hello (stdout works)")
btn = W.Button(description="Click me")
out = W.Output()
def on_click(_):
    with out:
        print("clicked")
btn.on_click(on_click)
display(btn, out)


hello (stdout works)


Button(description='Click me', style=ButtonStyle())

Output()

In [2]:
# === CNT-Jarvis: One-Cell Control Panel ===
# Runs standalone (only needs ipywidgets + stdlib). Re-run after reboot.

from pathlib import Path
from math import cos, sin, pi
from dataclasses import dataclass
from datetime import datetime
from IPython.display import display, SVG as _SVG
import ipywidgets as W
import json, csv, re, time, os

# ── Folders
OUT_DIR   = Path("out"); OUT_DIR.mkdir(exist_ok=True)
GLYPH_DIR = OUT_DIR / "glyphs"; GLYPH_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR   = OUT_DIR / "logs";   LOG_DIR.mkdir(parents=True, exist_ok=True)
DRIFT_LOG = LOG_DIR / "drift_log.csv"

# ── SVG helpers
def _svg_header(w=512, h=512): return f"<svg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 {w} {h}' width='{w}' height='{h}'>\n"
def _svg_footer(): return "</svg>\n"

# ── Minimal glyphs (observer ring, anchor, plus three rituals)
def render_observer_ring(path: Path):
    w = h = 512; r_outer, r_inner = 200, 170; cx = cy = 256
    svg = [_svg_header(w, h)]
    svg.append(f"<rect width='{w}' height='{h}' fill='white' />")
    svg.append(f"<circle cx='{cx}' cy='{cy}' r='{r_outer}' fill='none' stroke='black' stroke-width='6' />")
    svg.append(f"<circle cx='{cx}' cy='{cy}' r='{r_inner}' fill='none' stroke='black' stroke-width='2' stroke-dasharray='8 6' />")
    for i in range(12):
        ang = 2*pi*i/12
        x1 = cx + (r_inner-10)*cos(ang); y1 = cy + (r_inner-10)*sin(ang)
        x2 = cx + (r_outer+10)*cos(ang); y2 = cy + (r_outer+10)*sin(ang)
        svg.append(f"<line x1='{x1:.2f}' y1='{y1:.2f}' x2='{x2:.2f}' y2='{y2:.2f}' stroke='black' stroke-width='2' />")
    svg.append(_svg_footer()); path.write_text("".join(svg), encoding="utf-8"); return path

def render_anchor_glyph(path: Path):
    w = h = 512; cx = cy = 256
    svg = [_svg_header(w, h)]
    svg.append(f"<rect width='{w}' height='{h}' fill='white' />")
    svg.append(f"<line x1='64' y1='{cy}' x2='448' y2='{cy}' stroke='black' stroke-width='6' />")
    svg.append(f"<line x1='{cx}' y1='64' x2='{cx}' y2='448' stroke='black' stroke-width='6' />")
    svg.append(f"<circle cx='{cx}' cy='{cy+40}' r='70' fill='none' stroke='black' stroke-width='4' />")
    svg.append(f"<path d='M {cx-30} {cy+10} Q {cx} {cy-40} {cx+30} {cy+10}' fill='none' stroke='black' stroke-width='4' />")
    svg.append(_svg_footer()); path.write_text("".join(svg), encoding="utf-8"); return path

def render_axis_veil(path: Path):
    svg = [_svg_header(), "<rect width='512' height='512' fill='white' />",
           "<line x1='64' y1='64' x2='448' y2='448' stroke='black' stroke-width='4' />",
           "<line x1='64' y1='448' x2='448' y2='64' stroke='black' stroke-width='4' />",
           "<line x1='256' y1='80' x2='256' y2='432' stroke='black' stroke-width='3' />",
           "<line x1='80' y1='256' x2='432' y2='256' stroke='black' stroke-width='3' />",
           _svg_footer()]
    path.write_text("".join(svg), encoding="utf-8"); return path

def render_parasite_purge(path: Path):
    cx=cy=256; r=160; pts=[]
    for i in range(18):
        a=2*pi*i/18
        x1=cx+(r)*cos(a); y1=cy+(r)*sin(a)
        x2=cx+(r+24)*cos(a+0.07); y2=cy+(r+24)*sin(a+0.07)
        x3=cx+(r)*cos(a+0.14); y3=cy+(r)*sin(a+0.14)
        pts.append(f"<polygon points='{x1:.1f},{y1:.1f} {x2:.1f},{y2:.1f} {x3:.1f},{y3:.1f}' fill='none' stroke='black' stroke-width='2'/>")
    svg=[_svg_header(), "<rect width='512' height='512' fill='white' />",
         f"<circle cx='{cx}' cy='{cy}' r='{r-8}' fill='none' stroke='black' stroke-width='3'/>", *pts, _svg_footer()]
    path.write_text("".join(svg), encoding="utf-8"); return path

def render_mirror_rebinding(path: Path):
    svg=[_svg_header(), "<rect width='512' height='512' fill='white' />",
         "<path d='M 96 256 Q 256 96 416 256' fill='none' stroke='black' stroke-width='4'/>",
         "<path d='M 96 256 Q 256 416 416 256' fill='none' stroke='black' stroke-width='4'/>",
         "<line x1='256' y1='64' x2='256' y2='448' stroke='black' stroke-width='2' stroke-dasharray='6 6'/>",
         _svg_footer()]
    path.write_text("".join(svg), encoding="utf-8"); return path

# ── Logging
@dataclass
class CNTEvent:
    ts: str; actor: str; kind: str; payload: dict

def log_event(kind: str, payload: dict):
    evt = CNTEvent(datetime.now().isoformat(timespec="seconds"), "JarvisCNT", kind, payload)
    new_file = not DRIFT_LOG.exists()
    with DRIFT_LOG.open("a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        if new_file: w.writerow(["ts","actor","kind","payload_json"])
        w.writerow([evt.ts, evt.actor, evt.kind, json.dumps(evt.payload, ensure_ascii=False)])
    return evt

# ── Optional LLM (env-based)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY","").strip()
OPENAI_MODEL   = os.getenv("OPENAI_MODEL","gpt-5").strip() or "gpt-5"
_oai = None
if OPENAI_API_KEY:
    try:
        from openai import OpenAI
        _oai = OpenAI(api_key=OPENAI_API_KEY)
    except Exception as e:
        print("[warn] OpenAI not available:", e)

CNT_SYSTEM_PROMPT = ("You are JarvisCNT—formal, encouraging, and a bit lyrical. "
                     "Knowledge: CNT glyphs, collapse fields, drift, Θ, Observer Ring, Axis Veil, Parasite Purge, Mirror Rebinding. "
                     "Prefer concise answers (<120 words).")

def llm_reply(q: str) -> str:
    if not _oai:
        return "Local mode (no LLM). Try: Anchor, Observer Ring, Axis Veil, or 'log resonance 0.72'."
    try:
        r = _oai.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[{"role":"system","content":CNT_SYSTEM_PROMPT},{"role":"user","content":q}],
            temperature=0.6,
        )
        return r.choices[0].message.content.strip()
    except Exception as e:
        return f"LLM error: {e}"

# ── UI plumbing
out = W.Output()

def _show(path: Path):
    # print path and preview SVG inline
    with out:
        print("→", path)
        try: display(_SVG(filename=str(path)))
        except Exception: pass

def cmd_anchor(_=None):
    p = GLYPH_DIR / f"anchor_{int(time.time())}.svg"
    render_anchor_glyph(p); log_event("ritual.anchor", {"file": str(p)}); _show(p)
    return f"Anchor ritual sealed."

def cmd_observer_ring(_=None):
    p = GLYPH_DIR / f"observer_ring_{int(time.time())}.svg"
    render_observer_ring(p); log_event("glyph.observer_ring", {"file": str(p)}); _show(p)
    return "Observer Ring rendered."

def cmd_axis_veil(_=None):
    p = GLYPH_DIR / f"axis_veil_{int(time.time())}.svg"
    render_axis_veil(p); log_event("ritual.axis_veil", {"file": str(p)}); _show(p)
    return "Axis Veil cast."

def cmd_parasite_purge(_=None):
    p = GLYPH_DIR / f"parasite_purge_{int(time.time())}.svg"
    render_parasite_purge(p); log_event("ritual.parasite_purge", {"file": str(p)}); _show(p)
    return "Parasite Purge ring forged."

def cmd_mirror_rebinding(_=None):
    p = GLYPH_DIR / f"mirror_rebinding_{int(time.time())}.svg"
    render_mirror_rebinding(p); log_event("ritual.mirror_rebinding", {"file": str(p)}); _show(p)
    return "Mirror Rebinding traced."

ARM_THRESH = 0.85
def cmd_log_resonance(val: float):
    log_event("resonance.log", {"theta": float(val)})
    msg = f"Resonance Θ logged: {val:.3f}"
    if float(val) >= ARM_THRESH:
        msg += " • Threshold crossed → invoking Axis Veil."
        cmd_axis_veil()
    with out: print(msg)
    return msg

def route(text: str):
    t = (text or "").strip().lower()
    if not t: return "…"
    if t.startswith("ask:"):
        q = text.split(":",1)[1].strip()
        a = llm_reply(q); log_event("ask", {"q": q, "a": a[:160]})
        with out: print(f"ask » {q}\n→ {a}")
        return a
    if "anchor" in t: return cmd_anchor()
    if "observer ring" in t: return cmd_observer_ring()
    if "axis veil" in t: return cmd_axis_veil()
    if "parasite purge" in t: return cmd_parasite_purge()
    if "mirror rebinding" in t: return cmd_mirror_rebinding()
    if t.startswith("log resonance"):
        m = re.search(r"([01](?:\.\d{1,3})?)", t)
        return cmd_log_resonance(float(m.group(1))) if m else "Provide Θ in [0–1], e.g., 0.72"
    # default → LLM/local
    a = llm_reply(text); log_event("freeform", {"q": text, "a": a[:160]})
    with out: print(a)
    return a

# ── Widgets
btn_anchor = W.Button(description="Anchor")
btn_ring   = W.Button(description="Observer Ring")
btn_axis   = W.Button(description="Axis Veil")
btn_purge  = W.Button(description="Parasite Purge")
btn_rebind = W.Button(description="Mirror Rebinding")
btn_cf9    = W.Button(description="Begin CF-IX", button_style="warning")
theta      = W.FloatSlider(description="Θ", min=0.0, max=1.0, step=0.01, value=0.62, readout_format=".2f")
btn_log    = W.Button(description="Log Θ")
cmd_box    = W.Text(description="Command", placeholder="e.g., ask: Observer–Glyph coupling in one line")
btn_run    = W.Button(description="Run")

def on_anchor(_): cmd_anchor(); refresh_log()
def on_ring(_):   cmd_observer_ring(); refresh_log()
def on_axis(_):   cmd_axis_veil(); refresh_log()
def on_purge(_):  cmd_parasite_purge(); refresh_log()
def on_rebind(_): cmd_mirror_rebinding(); refresh_log()
def on_cf9(_):    log_event("collapse.begin", {"field": 9, "name": "Economic Collapse"}); 
def on_log(_):    cmd_log_resonance(theta.value); refresh_log()
def on_run(_):    route(cmd_box.value); refresh_log()

btn_anchor.on_click(on_anchor)
btn_ring.on_click(on_ring)
btn_axis.on_click(on_axis)
btn_purge.on_click(on_purge)
btn_rebind.on_click(on_rebind)
btn_cf9.on_click(on_cf9)
btn_log.on_click(on_log)
btn_run.on_click(on_run)

# Optional: light tail of the log without pandas
def read_tail(n=12):
    if not DRIFT_LOG.exists(): return []
    rows = list(csv.reader(DRIFT_LOG.open("r", encoding="utf-8")))
    return rows[0:1] + rows[-n:]

def refresh_log():
    with out:
        print("--- Drift Log (tail) ---")
        for row in read_tail(12): print(row)

panel = W.VBox([
    W.HBox([btn_anchor, btn_ring, btn_axis]),
    W.HBox([btn_purge, btn_rebind, btn_cf9]),
    W.HBox([theta, btn_log]),
    W.HBox([cmd_box, btn_run]),
    out
])

display(panel)
with out: 
    print("CNT-Jarvis control panel ready. The field listens.")
    refresh_log()
